### **TESTING MODEL WITH DATA**

We will use the Bir-directional LSTM method to test data because when the models were compared based on the Test loss and Test Accuracy. The Bidirectional LSTM performed slightly better than the GloVe model. . We use the Bidirectional LSTM to make the predictions for the tweets that will be used to infer election results.


In [19]:
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [17]:
congress_test = pd.read_csv('congress_test.csv')
bjp_test = pd.read_csv('bjp_test.csv')

In [18]:
congress_test =congress_test[:2000]
bjp_test = bjp_test[0:2000]

In [20]:
def tweet_to_words( raw_review ):
    # Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", str(raw_review))
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', str(letters_only)) # remove URLs
    tweet = re.sub('RT', ' ', str(tweet))

    #Convert to lower case, split into individual words
    tweet = letters_only.lower().split()



    return( " ".join(tweet))

In [21]:
# Get the number of Tweets based on the dataframe column size
num_tweets = 2000

# Initialize an empty list to hold the clean reviews


# Loop over each tweet; create an index i that goes from 0 to the length
# of the tweet list
def clean_test(dataframe):
    clean_train_tweets = []
    for i in range( 0, num_tweets ):
        # Call function for each one, and add the result to the list of
        clean_train_tweets.append( tweet_to_words(dataframe[i]))
    return clean_train_tweets

In [22]:
congress_inputs = clean_test(congress_test['clean_text'])
bjp_inputs = clean_test(bjp_test['clean_text'])

In [23]:
def tokenze_data(data_inputs):
        tokenizer = Tokenizer(nb_words=2000)
        tokenizer.fit_on_texts(data_inputs)
        sequences = tokenizer.texts_to_sequences(data_inputs)

        word_index = tokenizer.word_index
        print('Found %s unique tokens.' % len(word_index))
        max_len = 200
        data = pad_sequences(sequences, max_len)
        print('Shape of data tensor:', data.shape)
        indices = np.arange(data.shape[0])
        np.random.shuffle(indices)
        data = data[indices]
        return data

In [24]:
congress_inputs = tokenze_data(congress_inputs)
bjp_inputs = tokenze_data(bjp_inputs)

Found 3459 unique tokens.
Shape of data tensor: (2000, 200)
Found 2588 unique tokens.
Shape of data tensor: (2000, 200)


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/text.py:98: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn(


In [25]:
from keras.models import model_from_json

# load json and create model
with open("/content/Model_Bidir_LSTM.json", 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("/content/Weights_bidir_LSTM.weights.h5")
print("Loaded model from disk")


Loaded model from disk


In [26]:
congress_prediction = loaded_model.predict(congress_inputs)
bjp_prediction = loaded_model.predict(bjp_inputs)

63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step


In [27]:
congress_pred = (congress_prediction>0.5)
bjp_pred = (bjp_prediction>0.5)

In [28]:
def get_predictions(party_pred):
    x = 0
    for i in party_pred:
        if(i[1]==True):
            x+=1
    return x

In [29]:
congress_numbers = get_predictions(congress_pred)
bjp_numbers = get_predictions(bjp_pred)
print("Congress Positive Tweets:",congress_numbers)
print("BJP Positive Tweets:",bjp_numbers)

Congress Positive Tweets: 394
BJP Positive Tweets: 887
